In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/kaggle-llm-science-exam/sample_submission.csv
/kaggle/input/kaggle-llm-science-exam/train.csv
/kaggle/input/kaggle-llm-science-exam/test.csv
/kaggle/input/additional-train-data-for-llm-science-exam/6000_train_examples.csv
/kaggle/input/additional-train-data-for-llm-science-exam/extra_train_set.csv
/kaggle/input/test-with-infer/test_with_infer.csv
/kaggle/input/test-with-infer-15k/test_with_infer_15k.csv
/kaggle/input/deberta-v3-large-hf-weights/spm.model
/kaggle/input/deberta-v3-large-hf-weights/config.json
/kaggle/input/deberta-v3-large-hf-weights/tokenizer_config.json
/kaggle/input/deberta-v3-large-hf-weights/pytorch_model.bin
/kaggle/input/15k-high-quality-examples/5900_examples.csv
/kaggle/input/15k-high-quality-examples/15k_gpt3.5-turbo.csv
/kaggle/input/llm-whls/einops-0.6.1-py3-none-any.whl
/kaggle/input/llm-whls/tokenizers-0.14.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
/kaggle/input/llm-whls/trl-0.5.0-py3-none-any.whl
/kaggle/input/llm-whls/tran

# Data Process

In [2]:
!pip install --no-index --no-deps /kaggle/input/llm-whls/peft-0.4.0-py3-none-any.whl
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"

from typing import Optional, Union
import pandas as pd
import numpy as np
import torch
from datasets import Dataset
from dataclasses import dataclass
from transformers import AutoTokenizer
from transformers.tokenization_utils_base import PreTrainedTokenizerBase, PaddingStrategy
from transformers import AutoModelForMultipleChoice, TrainingArguments, Trainer, AutoModel
from peft import LoraConfig, get_peft_model, TaskType

deberta_v3_large= '/kaggle/input/deberta-v3-large-hf-weights'

Processing /kaggle/input/llm-whls/peft-0.4.0-py3-none-any.whl


2024-02-21 12:58:14.948784: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-21 12:58:14.948902: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-21 12:58:15.082656: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
test = pd.read_csv("/kaggle/input/15k-high-quality-examples/5900_examples.csv").dropna().reset_index(drop=True)
test = pd.concat([test.iloc[:400],pd.read_csv("/kaggle/input/kaggle-llm-science-exam/train.csv").drop("id", axis=1)])
test_infer = pd.read_csv("/kaggle/input/test-with-infer/test_with_infer.csv").iloc[:,:-2]
print(test.shape)
print(test_infer.shape)

(600, 7)
(600, 7)


In [4]:
test_infer.head()

,prompt,B,D,C,E,A,answer
0,"Too Much TV is a British television programme,...",Exploring the history and evolution of British...,Showcasing a wide range of television programm...,Investigating the effects of excessive televis...,Analyzing the impact of excessive television c...,Examining the different genres and formats of ...,D
1,The 1919–20 Navy Midshipmen men's basketball t...,1905,1945,1957,1920,1893,A
2,"The original series, , is collected in 20 volu...",Ten volumes,Three volumes,Seven volumes,One volume,Five volumes,B
3,Several qualitative observations can be made o...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,C
4,The A4113 road is a single-carriageway road th...,Powys and Herefordshire,Knighton and Bromfield,Powys and Shropshire,Herefordshire and Bromfield,Shropshire and Knighton,B


In [5]:
# validation = pd.read_csv('/kaggle/input/kaggle-llm-science-exam/train.csv')
# validation.dropna(inplace=True)
# validation.reset_index(inplace=True,drop=True)
validation = pd.read_csv("/kaggle/input/test-with-infer-15k/test_with_infer_15k.csv").iloc[:100,:-2]
print(validation.shape)
df_train = pd.read_csv("/kaggle/input/15k-high-quality-examples/15k_gpt3.5-turbo.csv")
df_train.dropna(inplace=True)
df_train.reset_index(inplace=True,drop=True)
df_train = pd.concate([pd.read_csv("/kaggle/input/test-with-infer-15k/test_with_infer_15k.csv").iloc[-100:,:-2], df_train])
print(df_train.shape)

(100, 7)
(14993, 7)


In [6]:
option_to_index = {option: idx for idx, option in enumerate('ABCDE')}
index_to_option = {v: k for k,v in option_to_index.items()}
tokenizer = AutoTokenizer.from_pretrained(deberta_v3_large)

def preprocess(example):
    first_sentence = [example['prompt']] * 5
    second_sentences = [example[option] for option in 'ABCDE']
    tokenized_example = tokenizer(first_sentence, second_sentences, truncation=False)
    tokenized_example['label'] = option_to_index[example['answer']]
    
    return tokenized_example

@dataclass
class DataCollatorForMultipleChoice:
    tokenizer: PreTrainedTokenizerBase
    padding: Union[bool, str, PaddingStrategy] = True
    max_length: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    
    def __call__(self, features):
        label_name = 'label' if 'label' in features[0].keys() else 'labels'
        labels = [feature.pop(label_name) for feature in features]
        batch_size = len(features)
        num_choices = len(features[0]['input_ids'])
        flattened_features = [
            [{k: v[i] for k, v in feature.items()} for i in range(num_choices)] for feature in features
        ]
        flattened_features = sum(flattened_features, [])
        
        batch = self.tokenizer.pad(
            flattened_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors='pt',
        )
        batch = {k: v.view(batch_size, num_choices, -1) for k, v in batch.items()}
        batch['labels'] = torch.tensor(labels, dtype=torch.int64)
        return batch

/opt/conda/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:515: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [7]:
dataset = Dataset.from_pandas(df_train)
print(dataset)
tokenized_dataset = dataset.map(preprocess, remove_columns=['prompt', 'A', 'B', 'C', 'D', 'E', 'answer'])
print(tokenized_dataset)

Dataset({
    features: ['prompt', 'A', 'B', 'C', 'D', 'E', 'answer'],
    num_rows: 14993
})


  0%|          | 0/14993 [00:00<?, ?ex/s]

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'label'],
    num_rows: 14993
})


In [8]:
tokenized_test = Dataset.from_pandas(test).map(preprocess, remove_columns=['prompt', 'A', 'B', 'C', 'D', 'E', 'answer'])
print(tokenized_test)
tokenized_test_infer = Dataset.from_pandas(test_infer).map(preprocess, remove_columns=['prompt', 'A', 'B', 'C', 'D', 'E', 'answer'])
print(tokenized_test_infer)
tokenized_validation = Dataset.from_pandas(validation).map(preprocess, remove_columns=['prompt', 'A', 'B', 'C', 'D', 'E', 'answer'])
print(tokenized_validation)

  0%|          | 0/600 [00:00<?, ?ex/s]

Dataset({
    features: ['__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask', 'label'],
    num_rows: 600
})


  0%|          | 0/600 [00:00<?, ?ex/s]

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'label'],
    num_rows: 600
})


  0%|          | 0/100 [00:00<?, ?ex/s]

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'label'],
    num_rows: 100
})


# Train

In [9]:
def map_at_3(predictions, labels):
    map_sum = 0
    pred = np.argsort(-1*np.array(predictions),axis=1)[:,:3]
    for x,y in zip(pred,labels):
        z = [1/i if y==j else 0 for i,j in zip([1,2,3],x)]
        map_sum += np.sum(z)
    return map_sum / len(predictions)


def compute_metrics(p):
    predictions = p.predictions.tolist()
    labels = p.label_ids.tolist()
    return {"map@3": map_at_3(predictions, labels)}

In [10]:
model = AutoModelForMultipleChoice.from_pretrained(deberta_v3_large)
# peft_config = LoraConfig(task_type=TaskType.TOKEN_CLS,r=8,
#                          lora_alpha=16,lora_dropout=0.1,
# #                          target_modules=["query_key_value"],
#                          modules_to_save=['classifier','pooler'])
# model = get_peft_model(model, peft_config)

FREEZE_LAYERS=6
print('Freezing embeddings.')
for param in model.deberta.embeddings.parameters():
    param.requires_grad = False
print(f'Freezing {FREEZE_LAYERS} layers.')
for layer in model.deberta.encoder.layer[:FREEZE_LAYERS]:
    for param in layer.parameters():
        param.requires_grad = False

training_args = TrainingArguments(
    learning_rate=1e-5,
    warmup_ratio=0.1,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=8,
    num_train_epochs=1,
    report_to='none',
    output_dir = f'./checkpoints',
    overwrite_output_dir=True,
    fp16=True,
    gradient_accumulation_steps=8,
    logging_steps=50,
    evaluation_strategy='steps',
    eval_steps=50,
    save_strategy="steps",
    save_steps=50,
    load_best_model_at_end=True,
    metric_for_best_model='map@3',
    lr_scheduler_type='cosine',
#     weight_decay=0.01,
    save_total_limit=1,
)
trainer = Trainer(
    model=model,
    args=training_args,
    tokenizer=tokenizer,
    data_collator=DataCollatorForMultipleChoice(tokenizer=tokenizer),
    train_dataset=tokenized_dataset,
    eval_dataset=tokenized_validation,
    compute_metrics = compute_metrics,
)
# model.print_trainable_parameters()

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of DebertaV2ForMultipleChoice were not initialized from the model checkpoint at /kaggle/input/deberta-v3-large-hf-weights and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Freezing embeddings.
Freezing 6 layers.


In [11]:
trainer.train()

You're using a DebertaV2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss,Map@3
50,1.614900,1.609219,0.446667
100,1.608700,1.608066,0.536667
150,1.497500,1.582773,0.671667
200,1.350400,1.559941,0.670000
250,1.305300,1.559121,0.711667
300,1.294200,1.562676,0.715000
350,1.252100,1.537060,0.728333
400,1.242400,1.542373,0.731667
450,1.250700,1.542139,0.735000


TrainOutput(global_step=468, training_loss=1.3736249157506177, metrics={'train_runtime': 2110.1687, 'train_samples_per_second': 7.105, 'train_steps_per_second': 0.222, 'total_flos': 7161916678487040.0, 'train_loss': 1.3736249157506177, 'epoch': 1.0})

In [12]:
model.save_pretrained("model_v1")

# Evaluation

In [13]:
import numpy as np
def precision_at_k(r, k):
    """Precision at k"""
    assert k <= len(r)
    assert k != 0
    return sum(int(x) for x in r[:k]) / k

def MAP_at_3(predictions, true_items):
    """Score is mean average precision at 3"""
    U = len(predictions)
    map_at_3 = 0.0
    for u in range(U):
        user_preds = predictions[u]
        user_true = true_items.tolist()[u]
        user_results = [1 if item == user_true else 0 for item in user_preds]
        for k in range(min(len(user_preds), 3)):
            map_at_3 += precision_at_k(user_results, k+1) * user_results[k]
    return map_at_3 / U

In [14]:
prediction = trainer.predict(tokenized_test).predictions
prediction_letter = np.array(list('ABCDE'))[np.argsort(-prediction, 1)]
MAP_at_3(prediction_letter, test["answer"])

0.7536111111111108

In [15]:
prediction_infer = trainer.predict(tokenized_test_infer).predictions
prediction_infer_letter = np.array(list('ABCDE'))[np.argsort(-prediction_infer, 1)]
MAP_at_3(prediction_infer_letter, test_infer["answer"])

0.8008333333333328

In [16]:
# trainable parameters
params = model.parameters()
num_trainable_params = 0
for param in model.parameters():
    if param.requires_grad:
        num_trainable_params += param.numel()

print(f"Number of trainable parameters: {num_trainable_params}")


Number of trainable parameters: 228308993


# Load Model

In [17]:
# from peft import PeftModel,PeftConfig
# # base_model_dir = "/kaggle/input/deberta-v3-large-hf-weights"
# # peft_model_dir = "/kaggle/working/model_v1"
# # test_model = AutoModelForMultipleChoice.from_pretrained(base_model_dir).cuda()
# # test_model = PeftModel.from_pretrained(test_model, peft_model_dir)
# model_dir = "/kaggle/working/model_v1"
# test_model = AutoModelForMultipleChoice.from_pretrained(model_dir).cuda()
# trainer = Trainer(test_model,tokenizer=tokenizer,
#     data_collator=DataCollatorForMultipleChoice(tokenizer=tokenizer))
# prediction = trainer.predict(tokenized_test).predictions
# prediction_letter = np.array(list('ABCDE'))[np.argsort(-prediction, 1)]

In [18]:
# MAP_at_3(prediction_letter, test["answer"])